In [36]:
import pandas as pd
import datetime as dt
import numpy as np
import csv

from numpy import sin, cos, sqrt

from astropy.time import Time

import pdir

import os

## Import

In [37]:
# Script input

# FILENAME = "horizons_results_earth_2019-2020.txt"
# FILENAME = "horizons_results_earth_2019-2039.txt"
# FILENAME = "horizons_results_earth_2019-2262.txt"

# FILENAME = "horizons_results_mars_2019-2020.txt"
FILENAME = "horizons_results_mars_2019-2039.txt"
# FILENAME = "horizons_results_mars_2019-2262.txt"

limited_timespan = True  # Set false if more years than 2019-2261 are wanted, but dates will not be converted

Assumed format of filename:

`horizons_results_[BODY]_[START_YEAR]-[END-YEAR].txt`

where the dates of START_YEAR (closed interval) and END_YEAR (open interval) are assumed to be 01-01 00:00:00 (i.e. January 1st 00:00:00).

Due to Timestamp conversion limitations, we don't allow dates above 2262-01-01:
```python
print(pd.Timestamp.min)  # 1677-09-21 00:12:43.145225
print(pd.Timestamp.max)  # 2262-04-11 23:47:16.854775807
```

If more years are needed, re-write script to now use `pd.to_datetime`, i.e. convert date to pandas type Timestamp.

NOTE:
Horizon limits the number of rows such that startin from 2019, 2265 is the maximum year anyway.
Also ephemerides are [available](https://ssd.jpl.nasa.gov/eph_spans.cgi?id=A) in the following time spans:

- **Earth:** B.C. 9998-Mar-20 to A.D. 9999-Dec-31
- **Mars:** 1600-Jan-01 to 2500-Jan-04 

In [38]:
# Change CDW (current working directiory)

FILE_PATH = "../orbsim/r4b_3d/ephemerides/"

cwd = os.getcwd()
in_correct_cwd = 'code' + FILE_PATH[2:-1] == cwd[-30:] # check if last part of cwd is '/code/orbsim/r4b_3d'

if not in_correct_cwd:    
    os.chdir(FILE_PATH)
    cwd = os.getcwd()

print(cwd)

/Users/gandalf/Dropbox/repositories/letomes/code/orbsim/r4b_3d/ephemerides


In [39]:
# Derived variables

_, _, BODY, DATE_RANGE = FILENAME.split('.')[0].split('_')
START_YEAR, END_YEAR = DATE_RANGE.split('-')
OUTPUT = "{}_{}.csv".format(BODY, DATE_RANGE)  # output filename

with open(FILENAME, 'r') as file:
    for i, line in enumerate(file):
        if line == "$$SOE\n":
            SOE = i + 1
        if line == "$$EOE\n":
            EOE = i + 1

FOOTER = 117
EOD = EOE + FOOTER

print("SOE: {}".format(SOE))
print("EOE: {}".format(EOE))
print("FOOTER: {}".format(FOOTER))
print("EOD: {}".format(EOD))

print("BODY: {}".format(BODY))
print("DATE_RANGE: {}".format(DATE_RANGE))
print("START_YEAR: {}".format(START_YEAR))
print("END_YEAR: {}".format(END_YEAR))
print("OUTPUT: {}".format(OUTPUT))
print("SOE: {}".format(SOE))

SOE: 62
EOE: 7369
FOOTER: 117
EOD: 7486
BODY: mars
DATE_RANGE: 2019-2039
START_YEAR: 2019
END_YEAR: 2039
OUTPUT: mars_2019-2039.csv
SOE: 62


In [40]:
# Check if date interval is valid

if int(START_YEAR) < 1678:
    raise ValueError("Ephemerides end date must be in a year no earier than 1678 (01-01), otherwise pd.to_datetime fails down below")

if int(END_YEAR) > 2262:
    raise ValueError("Ephemerides end date must be in a year no later than 2262 (01-01), otherwise pd.to_datetime fails down below")

In [41]:
# Read in raw txt file

df = pd.read_csv(FILENAME,
                 skiprows=[x for x in range(0,SOE) if x!=SOE-3], # skip all rows until SOE except for headers
                 skipfooter=FOOTER,
                 engine='python',
                 error_bad_lines=False,
                 quoting=csv.QUOTE_NONE)

df.rename(columns=lambda x: x.strip(), inplace=True)  # strip whitespace from headers
df.drop('Unnamed: 26', axis=1, inplace=True)  # dropped erroneous 'n.p.' column

df

,Date__(UT)__HR:MN:SC.fff,Date_________JDUT,,.1,R.A._(ICRF/J2000.0),DEC_(ICRF/J2000.0),dRA*cosD,d(DEC)/dt,SN.ang,SN.dist,...,delta,deldot,VmagSn,VmagOb,PlAng,ObsEcLon,ObsEcLat,GlxLon,GlxLat,Tru_Anom
0,2019-Jan-01 00:00:00.000,2458484.5,,,39.051247,15.003341,85.77751,-6.245750,94.16,0.0,...,1.453495,2.054069,25.26810,25.26810,-0.00910,41.399454,-0.260651,157.204852,-40.793811,65.1878
1,2019-Jan-02 00:00:00.000,2458485.5,,,39.609720,15.196937,85.63319,-6.292350,94.20,0.0,...,1.454684,2.063466,25.24835,25.24835,-0.00909,41.972059,-0.242347,157.677090,-40.347123,65.7607
2,2019-Jan-03 00:00:00.000,2458486.5,,,40.168299,15.388839,85.48861,-6.338020,94.24,0.0,...,1.455878,2.072641,25.22853,25.22853,-0.00909,42.543725,-0.224048,158.142344,-39.899275,66.3326
3,2019-Jan-04 00:00:00.000,2458487.5,,,40.726984,15.579034,85.34379,-6.382770,94.28,0.0,...,1.457078,2.081596,25.20864,25.20864,-0.00908,43.114450,-0.205758,158.600793,-39.450333,66.9036
4,2019-Jan-05 00:00:00.000,2458488.5,,,41.285777,15.767507,85.19876,-6.426580,94.31,0.0,...,1.458282,2.090329,25.18868,25.18868,-0.00907,43.684233,-0.187477,159.052616,-39.000358,67.4737
5,2019-Jan-06 00:00:00.000,2458489.5,,,41.844679,15.954244,85.05352,-6.469470,94.35,0.0,...,1.459492,2.098842,25.16865,25.16865,-0.00907,44.253072,-0.169209,159.497984,-38.549411,68.0429
6,2019-Jan-07 00:00:00.000,2458490.5,,,42.403688,16.139231,84.90811,-6.511440,94.39,0.0,...,1.460707,2.107135,25.14856,25.14856,-0.00906,44.820965,-0.150953,159.937064,-38.097551,68.6111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7299,2038-Dec-26 00:00:00.000,2465783.5,,,244.189182,-21.860505,79.27719,7.472342,84.62,0.0,...,1.510523,-2.267369,24.33811,24.33811,0.00403,246.164624,-0.531630,353.503115,20.262784,269.8998
7300,2038-Dec-27 00:00:00.000,2465784.5,,,244.750112,-21.967684,79.41277,7.507191,84.60,0.0,...,1.509213,-2.267312,24.35904,24.35904,0.00403,246.695717,-0.548003,353.789447,19.804593,270.4313


### NOTE OUTLIER
271 September 29th, crazy declanation value

## Extract & rename relevant columns, export

In [42]:
# Extract and renam relevant columns, then export

eph = pd.DataFrame()
# eph['MJD'] = [Time(x).mjd for x in list(eph['date'].astype(str))]  # using astropy to convert
eph['MJD'] = df['Date_________JDUT'] - 2400000.5  # using definition   (https://en.wikipedia.org/wiki/Julian_day#Variants)
eph['date'] =  pd.to_datetime(df['Date__(UT)__HR:MN:SC.fff'])

eph['days'] = list(eph['MJD']) - eph['MJD'][0]
eph.set_index('days', inplace=True)

eph['r'] =  df['r']
eph['theta'] = 90 - df['DEC_(ICRF/J2000.0)']
eph['phi'] = df['R.A._(ICRF/J2000.0)']

rs = list(eph['r'])
thetas = np.radians(list(eph['theta']))
phis = np.radians(list(eph['phi']))
eph['x'] = rs * sin(thetas) * cos(phis)
eph['y'] = rs * sin(thetas) * sin(phis)
eph['z'] = rs * cos(thetas)
# eph['r2'] = sqrt(eph['x']**2 + eph['y']**2 + eph['z']**2)  # sanity check

eph.to_csv(OUTPUT)

pd.set_option('max_row', 15)
eph

,MJD,date,r,theta,phi,x,y,z
days,,,,,,,,
0.0,58484.0,2019-01-01,1.453495,74.996659,39.051247,1.090280,0.884507,0.376274
1.0,58485.0,2019-01-02,1.454684,74.803063,39.609720,1.081506,0.895008,0.381327
2.0,58486.0,2019-01-03,1.455878,74.611161,40.168299,1.072627,0.905423,0.386344
3.0,58487.0,2019-01-04,1.457078,74.420966,40.726984,1.063645,0.915751,0.391324
4.0,58488.0,2019-01-05,1.458282,74.232493,41.285777,1.054562,0.925992,0.396266
5.0,58489.0,2019-01-06,1.459492,74.045756,41.844679,1.045378,0.936144,0.401170
6.0,58490.0,2019-01-07,1.460707,73.860769,42.403688,1.036095,0.946207,0.406036
...,...,...,...,...,...,...,...,...
7299.0,65783.0,2038-12-26,1.510523,111.860505,244.189182,-0.610391,-1.262047,-0.562440
